In [1]:
%reset -f 
from __future__ import absolute_import, print_function
import time
import numpy as np
import pyopencl as cl
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy import signal
from matplotlib import rcParams
from scipy import io
import os

import locale
os.environ["PYTHONIOENCODING"] = "utf-8"

In [ ]:
# parameters
K       = 0.666
Q       = 1.2
M0      = 2.0
A       = 120.0
NW      = 1.5
NH      = 4.5
E       = 1.5
Lambda1 = 0.03
Gamma   = 14.0
DB      = 0.1
DW      = 2.5
DH      = 4.0
P       = 300.0
RW      = 0.3
RH      = 0.8
f       = 0.01

q       = Q/K
nuw     = NW/M0
nuh     = NH/M0
alpha   = A/M0
eta     = E*K
gamma   = Gamma*K/M0
p       = P*Lambda1/(M0*M0) #
deltaw  = DW/DB
deltah  = DH*M0/(DB*Lambda1)
Rw      = RW
Rh      = RH


# Spatial and temporal setting
length      = 4096.0      # Length of the physical landscape
n           = 4096       # Size of the 2D grid
endtime     = 40.0    # end time 800.0
dT          = 0.00004     # 
nplot       = 101
nsteps      = np.ceil(endtime/nplot/dT).astype(int)    # number of time steps
dX          = length/n      # Spatial step size
dY          = length/n      # Spatial step size
Grid_Width  = n
Grid_Height = n
# %% Reporting in the simulation on the console
print(" Current grid dimensions: %d x %d cells\n" % (Grid_Width, Grid_Height));

# Setting up the OpenCL context
DeviceNr = 1   # 0 = GTX 960M
platform = cl.get_platforms()
Devices  = platform[0].get_devices()  # 0 = GPU
context  = cl.Context([Devices[DeviceNr]])
queue    = cl.CommandQueue(context)
mf       = cl.mem_flags # Memory flags are set
print(" Compute Device: %s\n" % Devices[DeviceNr].name)

# Initial condition
H        = p/(alpha*f+nuh)+np.zeros(n*n)
H_host   = H.astype(np.float64)
H_g      = cl.Buffer(context, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=H_host)

CoefH    = np.zeros(n*n)
CoefH_host   = CoefH.astype(np.float64)
CoefH_g      = cl.Buffer(context, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=CoefH_host)

W        = alpha*f*p/(nuw*(alpha*f+nuh))+np.zeros(n*n)
W_host   = W.astype(np.float64)
W_g      = cl.Buffer(context, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=W_host)

BB       = np.random.rand(n*n)
B        = np.where(BB<0.1,2,0)
B_host   = B.astype(np.float64)
B_g      = cl.Buffer(context, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=B_host)

# Load opencl function
with open('SpatialFunctions_iPy.cl', 'r',encoding='utf-8') as myfile:
   SpatialFunctions = myfile.read()

# List of parameters for the OpenCL kernel. Seperate with comma without spaces
# Setting up the parameters for the Kernel
PassVars="q,nuw,nuh,alpha,eta,gamma,p,deltaw,deltah,Rw,Rh,f,dX,dY,dT,Grid_Width,Grid_Height"
PassVals=eval(PassVars)
PassVars=PassVars.split(',')
Params=""
for ii in range(len(PassVals)):
    Params = Params+"#define " + PassVars[ii] + " " + str(PassVals[ii]) + " \n"

ComputeCode = """
///////////////////////////////////////////////////////////////////////////////
// Simulation kernel
///////////////////////////////////////////////////////////////////////////////   

__kernel void SimulationKernel (__global float* H, __global float* W, __global float* B, __global float* CoefH)
{
    size_t current  = get_global_id(0);
    size_t row      = floor((float)current/(float)Grid_Width);
    size_t column   = current%Grid_Width;
        CoefH[current] = H[current]*H[current];
        if (row > 0 && row < Grid_Width-1 && column > 0 && column < Grid_Height-1)
            {
             float dHdt = p-alpha*(B[current]+q*f)/(B[current]+q)*H[current]-nuh/(1+Rh*B[current])*H[current]+deltah*d2_dxy2(CoefH);
             float dWdt = alpha*(B[current]+q*f)/(B[current]+q)*H[current]-nuw/(1+Rw*B[current])*W[current]-gamma*B[current]*(1+eta*B[current])*(1+eta*B[current])*W[current]+deltaw*d2_dxy2(W);
             float dBdt = W[current]*(1+eta*B[current])*(1+eta*B[current])*B[current]*(1-B[current])-B[current]+d2_dxy2(B);
             H[current] = H[current] + dHdt*dT;
             W[current] = W[current] + dWdt*dT;
             B[current] = B[current] + dBdt*dT;
            }
            
        // HANDLE Boundaries
        else 
            {
             PeriodicBoundaries(H);
             PeriodicBoundaries(B);
             PeriodicBoundaries(W);
            }

} // End SimulationKernel
"""

program = cl.Program(context, Params + SpatialFunctions + ComputeCode).build()

from ipywidgets import FloatProgress
from IPython.display import display

# Setting up a progress bar for the simulation
print("Progress :");
PB = FloatProgress(min=0, max=nplot); display(PB) 

#Ss=np.zeros((Grid_Width, Grid_Height, nplot))
#Bs=np.zeros((Grid_Width, Grid_Height, nplot))

# Set up simulation parameters
global_size = H_host.shape

# Start the timer:
start_time = time.time()

# Starting the loop
for ii in range(1,nplot):
    # The simulation
    for jj in range(1,nsteps):      
        program.SimulationKernel(queue, global_size, None, H_g, W_g, B_g, CoefH_g)

    # Get the data from the GPU
    cl.enqueue_copy(queue, H_host, H_g)
    cl.enqueue_copy(queue, B_host, B_g)
    
    # We store the state of the system for <NumPlot> different times.
    Hs = H_host.reshape(Grid_Width, Grid_Height)
    Bs = B_host.reshape(Grid_Width, Grid_Height)
    io.savemat('test/Arid_'+str(ii)+'.mat',{'B':Bs})
    PB.value += 1 # signal to increment the progress bar

# Determining the time that we used for the simulation
elapsed_time = time.time() - start_time    
print(" Simulation took      : %1.1f (s)" % (elapsed_time))

 Current grid dimensions: 4096 x 4096 cells

 Compute Device: AMD Radeon Pro Vega 56 Compute Engine

Progress :


FloatProgress(value=0.0, max=101.0)

In [ ]:
Bs = B_host.reshape(Grid_Width,Grid_Height)
plt.imshow(Bs)
plt.colorbar()

In [ ]:
print(np.max(B_host))

from scipy import io

io.savemat('data.mat',{'B':Bs})

In [ ]:
print(p)

In [ ]:
P       = 300.0
RW      = 0.3
RH      = 0.8
f       = 0.01

q       = Q/K
nuw     = NW/M0
nuh     = NH/M0
alpha   = A/M0
eta     = E*K
gamma   = Gamma*K/M0
p       = P*Lambda1/(M0*M0) #
deltaw  = DW/DB
deltah  = DH*M0/(DB*Lambda1)
Rw      = RW
Rh      = RH
print(p)

In [1]:
print('p'+str(1.0))

p1.0
